In [1]:
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data
import os
from prepare import clean_data,train_validate_test_split,split_titanic_data
from acquire import get_titanic_data 
from prepare import clean_titanic
from env import host,db,protocol,password,user,mysqlcon
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
import acquire
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
import env
from sklearn.neighbors import KNeighborsClassifier

Exercises

In these exercises, we'll continue working with the titanic dataset and building logistic regression models. Throughout this exercise, be sure you are training, evaluation, and comparing models on the train and validate datasets. The test dataset should only be used for your final model.

For all of the models you create, choose a threshold that optimizes for accuracy.

Create a new notebook, logistic_regression, use it to answer the following questions:

Create a model that includes only age, fare, and pclass. Does this model perform better than your baseline?

Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

Try out other combinations of features and models.

Use you best 3 models to predict and evaluate on your validate sample.

Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?

In [2]:
df = get_titanic_data()

In [3]:
df

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,886,0,2,male,27.0,0,0,13.0000,S,Second,NaN,Southampton,1
887,887,887,1,1,female,19.0,0,0,30.0000,S,First,B,Southampton,1
888,888,888,0,3,female,NaN,1,2,23.4500,S,Third,NaN,Southampton,0
889,889,889,1,1,male,26.0,0,0,30.0000,C,First,C,Cherbourg,1


In [4]:
clean_titanic(df)

,survived,pclass,age,sibling_spouse,parch,fare,alone,sex_male,embark_town_queenstown,embark_town_southampton,embark_town_southhampton
0,0,3,22.000000,1,0,7.2500,0,1,0,1,0
1,1,1,38.000000,1,0,71.2833,0,0,0,0,0
2,1,3,26.000000,0,0,7.9250,1,0,0,1,0
3,1,1,35.000000,1,0,53.1000,0,0,0,1,0
4,0,3,35.000000,0,0,8.0500,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,39.000000,0,5,29.1250,0,0,1,0,0
887,1,1,19.000000,0,0,30.0000,1,0,0,1,0
888,0,3,29.699118,1,2,23.4500,0,0,0,1,0
889,1,1,26.000000,0,0,30.0000,1,1,0,0,0


In [5]:
df = clean_titanic(df)

In [6]:
def train_validate_test_split(df, target='survived', seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.7, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.8, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test

In [7]:
# split into train, validate, test
train, validate, test = train_validate_test_split(df, target='survived', seed=123)

# create X & y version of train, where y is a series with just the target variable and X are all the features. 

X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived


In [8]:
y_train.mode().values

array([0])

In [9]:
baseline_preds = [0 for i in range(0, len(train))]

In [10]:
len([0 for i in range(0, len(train))]) == len(train)

True

In [11]:
baseline_accuracy = (y_train == 0).mean()

In [12]:
baseline_accuracy

0.5869565217391305

Create a model that includes only age, fare, and pclass. Does this model perform better than your baseline?
baseline = 0.5871559633027523
subset model = 0.7064220183486238

In [13]:
subset = ['age', 'fare', 'pclass']
X_train[subset]

,age,fare,pclass
91,20.000000,7.8542,3
834,18.000000,8.3000,3
94,59.000000,7.2500,3
831,0.830000,18.7500,2
516,34.000000,10.5000,2
596,29.699118,33.0000,2
446,13.000000,19.5000,2
40,40.000000,9.4750,3
60,22.000000,7.2292,3
538,29.699118,14.5000,3


In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
# make our thing:
lr = LogisticRegression()
lr.fit(X_train[subset], y_train)
lr.score(X_train[subset], y_train)

0.8260869565217391

Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

In [16]:
subset1 = ['age', 'fare', 'pclass', 'sex_male']
X_train[subset1]

,age,fare,pclass,sex_male
91,20.000000,7.8542,3,1
834,18.000000,8.3000,3,1
94,59.000000,7.2500,3,1
831,0.830000,18.7500,2,1
516,34.000000,10.5000,2,0
596,29.699118,33.0000,2,0
446,13.000000,19.5000,2,0
40,40.000000,9.4750,3,0
60,22.000000,7.2292,3,1
538,29.699118,14.5000,3,1


In [17]:
lr2 = LogisticRegression()
lr2.fit(X_train[subset1], y_train)
lr2.score(X_train[subset1], y_train)

0.8260869565217391

Try out other combinations of features and models.

In [18]:
subset2 = ['age', 'fare', 'pclass', 'sex_male', 'alone']
X_train[subset2]

,age,fare,pclass,sex_male,alone
91,20.000000,7.8542,3,1,1
834,18.000000,8.3000,3,1,1
94,59.000000,7.2500,3,1,1
831,0.830000,18.7500,2,1,0
516,34.000000,10.5000,2,0,1
596,29.699118,33.0000,2,0,1
446,13.000000,19.5000,2,0,0
40,40.000000,9.4750,3,0,0
60,22.000000,7.2292,3,1,1
538,29.699118,14.5000,3,1,1


In [19]:
lr3 = LogisticRegression()
lr3.fit(X_train[subset2], y_train)
lr3.score(X_train[subset2], y_train)

0.8260869565217391

Use you best 3 models to predict and evaluate on your validate sample.

In [20]:
X_val, y_val = X_validate, y_validate

In [21]:
lr3.score(X_val[subset2], y_val)

0.723404255319149

In [22]:
lr2.score(X_val[subset1], y_val)

0.7287234042553191

In [23]:
lr.score(X_val[subset], y_val)

0.6595744680851063

In [24]:
print(f'''
logreg model 1 validate score: {lr.score(X_val[subset], y_val)}
logreg model 2 validate score: {lr2.score(X_val[subset1], y_val)}
logreg model 3 validate score: {lr3.score(X_val[subset2], y_val)}''')


logreg model 1 validate score: 0.6595744680851063
logreg model 2 validate score: 0.7287234042553191
logreg model 3 validate score: 0.723404255319149


Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?

In [25]:
# if i were to finish here,
# i would pick lr3 as my best model
# and use it once on test to conclude my pipeline:
lr2.score(X_test[subset1], y_test)

0.7344322344322345

In [26]:
print(f'Test:{lr2.score(X_test[subset1], y_test)} Train: {lr2.score(X_train[subset1], y_train)} Validate:{lr2.score(X_val[subset1], y_val)}')

Test:0.7344322344322345 Train: 0.8260869565217391 Validate:0.7287234042553191
